# LLM Optimization Modelling Experiment

In [661]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [674]:
problem = '''You are task with maximizing the water flow in a network of pipes over 10 time steps. The water enters the system at point A and exits the system at point G. Hence, you need to maximize the amount of water that flows to point G. The water enters point A according to the function f(t) = max(-4/30t^3+t^2-0.234t+3, 0), where t is the time step. There are serveral points that are connected through pipes. Each pipe has a fixed capacity. Each point has to possibility to release water out of the system in case the outgoing pipes are full. There are 12 available pipes in total but only 10 of them can be activated. Deactivated pipes can not be used to transport water. The activation status of a pipe is set before any water arrives at point A and does not change. Finally, water can not be stored at the connection points which means that any water coming in at time t needs to be flowing out at time t + 1.

These are the pipe capacities:
AB: 3
AC: 6
AF: 1
BC: 3
BD: 10
BE: 4
CE: 4
CF: 4
DE: 5
EG: 5
FB: 12
FG: 7'''

## 2. Generate the mathematical model

In [692]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [693]:
#Show the resopnse in a formatted way
Markdown(response.text)

Let's define the following:

**Sets:**

*  $T$: Set of time steps, $T = \{1, 2, ..., 10\}$
*  $N$: Set of nodes in the network, $N = \{A, B, C, D, E, F, G\}$
*  $P$: Set of pipes, $P = \{AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG\}$

**Parameters:**

*  $c_{ij}$: Capacity of pipe $(i,j) \in P$, as given in the problem statement
* $f(t)$: Inflow function at point A, $f(t) = \text{max}(-4/30t^3 + t^2 - 0.234t + 3, 0)$

**Decision Variables:**

*  $x_{ij}(t)$: Flow through pipe $(i,j) \in P$ at time $t \in T$
*  $y_{ij}$: Binary variable, equal to 1 if pipe $(i,j) \in P$ is activated, 0 otherwise
*  $r_i(t)$: Amount of water released at node $i \in N$ at time $t \in T$ 

**Mathematical Model:**

**Objective Function:** Maximize the total flow out of point G over all time steps:
  
  Maximize $\sum_{t \in T} x_{EG}(t)$

**Constraints:**

1. **Flow conservation at each node:** For each node $i \in N$ and time $t \in T$:
    * Inflow + inflow from pipes = outflow to pipes + release
    * $\sum_{(j,i) \in P} x_{ji}(t) + (f(t) \text{ if } i = A) = \sum_{(i,j) \in P} x_{ij}(t) + r_i(t)$
2. **Pipe capacity:**  For each pipe $(i,j) \in P$ and time $t \in T$:
    * Flow through pipe is less than or equal to its capacity if the pipe is activated
    * $x_{ij}(t) \leq c_{ij} * y_{ij}$ 
3. **Activation limit:**
    * Only 10 pipes can be activated: 
    * $\sum_{(i,j) \in P} y_{ij} = 10$
4. **Non-negativity and binary constraints:**
    * $x_{ij}(t) \geq 0 \text{  } \forall (i,j) \in P, t \in T$
    * $r_i(t) \geq 0 \text{  } \forall i \in N, t \in T$
    * $y_{ij} \in \{0,1\} \text{  } \forall (i,j) \in P$

This model represents the problem of maximizing water flow in the given network. The objective function aims to maximize the outflow at point G. The constraints ensure flow balance at each node, respect pipe capacities, limit the number of activated pipes, and enforce non-negativity and binary restrictions on the variables. 


## 3. Generate the pyomo code

In [694]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [695]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data
c = {
    'AB': 5, 'AC': 3, 'AF': 2, 'BC': 4, 'BD': 2, 'BE': 3, 'CE': 1, 'CF': 4, 'DE': 2, 'EG': 6, 'FB': 3, 'FG': 5 
}  # Capacity of each pipe

# Define sets
T = set(range(1, 11))  # Set of time steps
N = {'A', 'B', 'C', 'D', 'E', 'F', 'G'}  # Set of nodes
P = {('A', 'B'), ('A', 'C'), ('A', 'F'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'E'), ('C', 'F'), ('D', 'E'), ('E', 'G'), ('F', 'B'), ('F', 'G')}  # Set of pipes

# Create model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(P, T, within=pyo.NonNegativeReals)  # Flow through each pipe at each time step
model.y = pyo.Var(P, within=pyo.Binary)  # 1 if pipe is activated, 0 otherwise
model.r = pyo.Var(N, T, within=pyo.NonNegativeReals)  # Amount of water released at each node at each time step

# Define objective function
def objective_rule(model):
    return sum(model.x['E', 'G', t] for t in T)  # Maximize flow out of point G
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Define constraints
def flow_balance_rule(model, i, t):
    inflow = sum(model.x[j, i, t] for j in N if (j, i) in P)
    if i == 'A':
        inflow += max(-4/30*t**3 + t**2 - 0.234*t + 3, 0)  # Inflow function at point A
    outflow = sum(model.x[i, j, t] for j in N if (i, j) in P) + model.r[i, t]
    return inflow == outflow
model.flow_balance = pyo.Constraint(N, T, rule=flow_balance_rule)  # Flow conservation at each node

def capacity_rule(model, i, j, t):
    return model.x[i, j, t] <= c[i+j] * model.y[i, j]  # Pipe capacity constraint
model.capacity = pyo.Constraint(P, T, rule=capacity_rule) 

def activation_limit_rule(model):
    return sum(model.y[i, j] for (i, j) in P) == 10  # Limit on activated pipes
model.activation_limit = pyo.Constraint(rule=activation_limit_rule) 

# Solve the model
solver = pyo.SolverFactory('glpk')  # Use GLPK solver
solver.solve(model)

# Print results
print("Optimal Solution:")
for t in T:
    print(f"Time Step {t}:")
    for (i, j) in P:
        if pyo.value(model.x[i, j, t]) > 0:
            print(f"Flow from {i} to {j}: {pyo.value(model.x[i, j, t])}")
    for i in N:
        if pyo.value(model.r[i, t]) > 0:
            print(f"Release at {i}: {pyo.value(model.r[i, t])}")

print(f"Total flow out of G: {pyo.value(model.objective)}")
```

This code defines the sets, parameters, decision variables, objective function, and constraints of the mathematical model. It then uses the GLPK solver to find the optimal solution that maximizes the total flow out of point G.  I used sample data for the capacity of each pipe, denoted by the `c` dictionary.


## 4. Input problem data and try running the generated code

In [696]:
import pyomo.environ as pyo

# Sample Data
c = {
    'AB': 5, 'AC': 3, 'AF': 2, 'BC': 4, 'BD': 2, 'BE': 3, 'CE': 1, 'CF': 4, 'DE': 2, 'EG': 6, 'FB': 3, 'FG': 5 
}  # Capacity of each pipe

# Define sets
T = set(range(1, 11))  # Set of time steps
N = {'A', 'B', 'C', 'D', 'E', 'F', 'G'}  # Set of nodes
P = {('A', 'B'), ('A', 'C'), ('A', 'F'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'E'), ('C', 'F'), ('D', 'E'), ('E', 'G'), ('F', 'B'), ('F', 'G')}  # Set of pipes

# Create model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(P, T, within=pyo.NonNegativeReals)  # Flow through each pipe at each time step
model.y = pyo.Var(P, within=pyo.Binary)  # 1 if pipe is activated, 0 otherwise
model.r = pyo.Var(N, T, within=pyo.NonNegativeReals)  # Amount of water released at each node at each time step

# Define objective function
def objective_rule(model):
    return sum(model.x['E', 'G', t] for t in T)  # Maximize flow out of point G
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Define constraints
def flow_balance_rule(model, i, t):
    inflow = sum(model.x[j, i, t] for j in N if (j, i) in P)
    if i == 'A':
        inflow += max(-4/30*t**3 + t**2 - 0.234*t + 3, 0)  # Inflow function at point A
    outflow = sum(model.x[i, j, t] for j in N if (i, j) in P) + model.r[i, t]
    return inflow == outflow
model.flow_balance = pyo.Constraint(N, T, rule=flow_balance_rule)  # Flow conservation at each node

def capacity_rule(model, i, j, t):
    return model.x[i, j, t] <= c[i+j] * model.y[i, j]  # Pipe capacity constraint
model.capacity = pyo.Constraint(P, T, rule=capacity_rule) 

def activation_limit_rule(model):
    return sum(model.y[i, j] for (i, j) in P) == 10  # Limit on activated pipes
model.activation_limit = pyo.Constraint(rule=activation_limit_rule) 

# Solve the model
solver = pyo.SolverFactory('glpk')  # Use GLPK solver
solver.solve(model)

# Print results
print("Optimal Solution:")
for t in T:
    print(f"Time Step {t}:")
    for (i, j) in P:
        if pyo.value(model.x[i, j, t]) > 0:
            print(f"Flow from {i} to {j}: {pyo.value(model.x[i, j, t])}")
    for i in N:
        if pyo.value(model.r[i, t]) > 0:
            print(f"Release at {i}: {pyo.value(model.r[i, t])}")

print(f"Total flow out of G: {pyo.value(model.objective)}")

Optimal Solution:
Time Step 1:
Flow from A to C: 0.632666666666667
Flow from A to B: 3.0
Flow from B to E: 3.0
Flow from C to E: 0.632666666666667
Flow from E to G: 3.63266666666667
Release at G: 3.63266666666667
Time Step 2:
Flow from B to D: 1.46533333333333
Flow from D to E: 1.46533333333333
Flow from A to C: 1.0
Flow from A to B: 4.46533333333333
Flow from B to E: 3.0
Flow from C to E: 1.0
Flow from E to G: 5.46533333333333
Release at G: 5.46533333333333
Time Step 3:
Flow from B to D: 2.0
Flow from D to E: 2.0
Flow from A to C: 1.0
Flow from A to B: 5.0
Flow from B to E: 3.0
Flow from C to E: 1.0
Flow from E to G: 6.0
Release at G: 6.0
Release at A: 1.698
Time Step 4:
Flow from B to D: 2.0
Flow from D to E: 2.0
Flow from A to C: 1.0
Flow from A to B: 5.0
Flow from B to E: 3.0
Flow from C to E: 1.0
Flow from E to G: 6.0
Release at G: 6.0
Release at A: 3.53066666666667
Time Step 5:
Flow from B to D: 2.0
Flow from D to E: 2.0
Flow from A to C: 1.0
Flow from A to B: 5.0
Flow from B to 

In [697]:
model.objective()

37.72666666666667

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [698]:
print(response.text)

Let's define the following:

**Sets:**

*  $T$: Set of time steps, $T = \{1, 2, ..., 10\}$
*  $N$: Set of nodes in the network, $N = \{A, B, C, D, E, F, G\}$
*  $P$: Set of pipes, $P = \{AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG\}$

**Parameters:**

*  $c_{ij}$: Capacity of pipe $(i,j) \in P$, as given in the problem statement
* $f(t)$: Inflow function at point A, $f(t) = \text{max}(-4/30t^3 + t^2 - 0.234t + 3, 0)$

**Decision Variables:**

*  $x_{ij}(t)$: Flow through pipe $(i,j) \in P$ at time $t \in T$
*  $y_{ij}$: Binary variable, equal to 1 if pipe $(i,j) \in P$ is activated, 0 otherwise
*  $r_i(t)$: Amount of water released at node $i \in N$ at time $t \in T$ 

**Mathematical Model:**

**Objective Function:** Maximize the total flow out of point G over all time steps:
  
  Maximize $\sum_{t \in T} x_{EG}(t)$

**Constraints:**

1. **Flow conservation at each node:** For each node $i \in N$ and time $t \in T$:
    * Inflow + inflow from pipes = outflow to pipes + release
    *

In [699]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data
c = {
    'AB': 5, 'AC': 3, 'AF': 2, 'BC': 4, 'BD': 2, 'BE': 3, 'CE': 1, 'CF': 4, 'DE': 2, 'EG': 6, 'FB': 3, 'FG': 5 
}  # Capacity of each pipe

# Define sets
T = set(range(1, 11))  # Set of time steps
N = {'A', 'B', 'C', 'D', 'E', 'F', 'G'}  # Set of nodes
P = {('A', 'B'), ('A', 'C'), ('A', 'F'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'E'), ('C', 'F'), ('D', 'E'), ('E', 'G'), ('F', 'B'), ('F', 'G')}  # Set of pipes

# Create model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(P, T, within=pyo.NonNegativeReals)  # Flow through each pipe at each time step
model.y = pyo.Var(P, within=pyo.Binary)  # 1 if pipe is activated, 0 otherwise
model.r = pyo.Var(N, T, within=pyo.NonNegativeReals)  # Amount of water released at each node at each time step

# Define objective function
def objective_rule(model):
    return sum(model.x['E', 'G', t] for t in T)  # Maximize flow out of point G
model.objective = pyo.Obje